In [1]:

import sqlite3
import pandas as pd
DATABASE_NAME="test_template.db"
def get_metadata(path=None,con=None):
    if path:
        con=sqlite3.connect(path)
    cur=con.cursor()
    sql="""
        SELECT
      m.name AS table_name, 
      p.cid AS col_id,
      p.name AS col_name,
      p.type AS col_type,
      p.pk AS col_is_pk,
      p.dflt_value AS col_default_val,
      p.[notnull] AS col_is_not_null
    FROM sqlite_master m
    LEFT OUTER JOIN pragma_table_info((m.name)) p
      ON m.name <> p.name
    WHERE m.type = 'table'
    ORDER BY table_name, col_id


    """
    cur.execute(sql)
    res= cur.fetchall()
    df=pd.DataFrame(res,columns=["table","col_id","col_name","col_type","col_pk","col_def","col_notnull"])
    def get_fkl(table):
        sql=f"PRAGMA foreign_key_list({table});"
        cur.execute(sql)
        res=cur.fetchall()
        return res
    tables=df["table"].unique()
    fk_dict_list=[]
    for table in tables:    
        res=get_fkl(table)
        for element in res:
            fk_dict={}
            fk_dict["table"]=table
            fk_dict["col_name"]=element[3]
            fk_dict["table_name_to"]=element[2]
            fk_dict["col_name_to"]=element[4]
            fk_dict["on_update"]=element[5]
            fk_dict["on_delete"]=element[6]
            fk_dict_list.append(fk_dict)
    dffk=pd.DataFrame(fk_dict_list)
    df=pd.merge(df,dffk,how='left',on=["table","col_name"])
    return df

def recreate_from_text(text_input_path,sqlite_output_path):
    with open(text_input_path) as f:
        sql_file = f.read()    
    con = sqlite3.connect(sqlite_output_path)
    cur= con.cursor()
    for sql in sql_file.split(";"):
        cur.execute(sql)
    con.commit()
    return con

if input("Recreate database? ")=="yes":
    print("recreating database")
    con=recreate_from_text("dbspecs.sql",":memory:")
    con_dest= sqlite3.connect(DATABASE_NAME)
    con.backup(con_dest)
    con.close()

con= sqlite3.connect(DATABASE_NAME)
df=get_metadata(con=con)
con.close()
list(df["table"].unique())

Recreate database? yes
recreating database


['answer', 'question', 'test', 'test_category', 'test_question']

In [2]:
df.head(30)

,table,col_id,col_name,col_type,col_pk,col_def,col_notnull,table_name_to,col_name_to,on_update,on_delete
0,answer,0,answer_id,INTEGER,1,None,1,NaN,NaN,NaN,NaN
1,answer,1,default_sequence,SMALLINT,0,None,1,NaN,NaN,NaN,NaN
2,answer,2,question_id,INTEGER,0,None,1,question,question_id,NO ACTION,SET NULL
3,answer,3,correct_flag,BOOLEAN,0,None,0,NaN,NaN,NaN,NaN
4,answer,4,correct_sequence,SMALLINT,0,None,1,NaN,NaN,NaN,NaN
5,answer,5,answer_list_id,TEXT,0,None,0,NaN,NaN,NaN,NaN
6,answer,6,answer_text,TEXT,0,None,1,NaN,NaN,NaN,NaN
7,answer,7,answer_notes,TEXT,0,None,0,NaN,NaN,NaN,NaN
8,question,0,question_id,INTEGER,1,None,1,NaN,NaN,NaN,NaN
9,question,1,question_text,TEXT,0,None,0,NaN,NaN,NaN,NaN


In [ ]:
import re
from  pprint import pprint
questions=[]
while True:
    question={}
    question_text=input("Question: ")
    if not question_text:
        break
    question["question_text"]=str.strip(str(question_text))
    n=0
    options=[]
    print("Enter responses, start with ! for the correct ones")
    print("or enter the sequence at the end ")
    while True:
        n+=1
        correct_flag_set=False
        option_dict={}
        option_text=input("Option "+str(n)+":")
        if not option_text:
            break
        print(option_text)
        if option_text[0]=="!":
            option_dict["correct_flag"]=True
            correct_flag_set=True
            option_text=option_text[1:]
        else:
            option_dict["correct_flag"]=False
        m= re.findall(r"(^\w?)(?:\) |\. ]) ?(.+)$",option_text)
        
        if m:
            submatches=m[0]
            option_dict["answer_list_id"]=submatches[0].strip()
            option_dict["answer_text"]=submatches[1].strip()
        else:
            option_dict["answer_list_id"]=str(n)
            option_dict["answer_text"]=option_text.strip()
        options.append(option_dict)
    correct_sequence=[]
    if not correct_flag_set:
        correct_sequence_text=input("Enter the correct response or sequence using the number and separated by spaces: ")
        if correct_sequence_text:
            correct_sequence_list=split(correct_sequence_text," ")
            correct_sequence_list=[int(n) for n in correct_sequence_list]
            pprint(correct_sequence_list)
            if len(correct_sequence_list)==0:
                options[0]["correct_flag"]=True
                
    question["answer"]=options.copy()
    questions.append(question)
pprint(questions)

In [6]:
import sqlite3
DATABASE_NAME="test_template.db"
con= sqlite3.connect(DATABASE_NAME)
cur=con.cursor()
def get_next_id(con,col_name,table_name):
    cur=con.cursor()
    cur.execute(f"SELECT MAX({col_name}) AS max_value FROM {table_name}",)
    res=cur.fetchone()[0]
    if res:
        next_id=int(res)+1
    else:
        next_id=1
    cur.close()
    return next_id

get_next_id(con,"question_id","question")

1

In [7]:

for question in questions:
    print(question)
    question_id=get_next_id(con,"question_id","question")
    sql = ''' INSERT INTO question ("question_id", "question_text", "question_notes") VALUES(?,?,?) '''
    values=(question_id,question["question_text"],"")
    cur.execute(sql,values)
    con.commit()

{'question_text': 'how are you today?', 'answer': [{'correct_flag': False, 'answer_list_id': '1', 'answer_text': 'Ok'}, {'correct_flag': False, 'answer_list_id': '2', 'answer_text': 'spectacular'}, {'correct_flag': False, 'answer_list_id': '3', 'answer_text': 'so so'}]}
{'question_text': 'What day is today?', 'answer': [{'correct_flag': False, 'answer_list_id': '1', 'answer_text': 'weekend!'}, {'correct_flag': False, 'answer_list_id': '2', 'answer_text': 'tgif'}, {'correct_flag': False, 'answer_list_id': '3', 'answer_text': 'meh'}]}


In [8]:
df.head()

,table,col_id,col_name,col_type,col_pk,col_def,col_notnull,table_name_to,col_name_to,on_update,on_delete
0,answer,0,answer_id,INTEGER,1,None,1,NaN,NaN,NaN,NaN
1,answer,1,default_sequence,SMALLINT,0,None,1,NaN,NaN,NaN,NaN
2,answer,2,question_id,INTEGER,0,None,1,question,question_id,NO ACTION,SET NULL
3,answer,3,correct_flag,BOOLEAN,0,None,0,NaN,NaN,NaN,NaN
4,answer,4,correct_sequence,SMALLINT,0,None,1,NaN,NaN,NaN,NaN


In [9]:
sql="""select * from question"""
cur.execute(sql)
res=cur.fetchall()
col_list= df[df["table"]=="question"]["col_name"].to_list()
dftemp=pd.DataFrame(res,columns=col_list)
dftemp

,question_id,question_text,question_notes
0,1,how are you today?,
1,2,What day is today?,
